In [7]:
import pandas as pd
from py2neo import Graph, Node, Relationship
import re # for time processing
 
# Thay đổi default path thành path đến folder chứa data set của 
DEFAULT_PATH = "/Users/nguyenvanviet/Work/Courses/DataGen/RecommandationCourseWeb-Neo4j/MyPaperDataSet/ETLCode/CleanData/" 

COMPETENCIES_LIST = ["Knowledge", "Platform", "Framework", "ProgrammingLanguage", "Tool"]


## connect to neo4j (CareerInsightDW2 - for paper || ThesisDW for thesis) 

In [3]:
uri = "neo4j+s://73f9a4e9.databases.neo4j.io:7687"  # thay đổi bolt tương ứng trong neo4j
username = "neo4j" # thay đổi bằng tên sử dụng trong neo4j
password = "XtvZQ_Fof-5SAkwa2eCEYQVNsTJ3xSqOaMM_X4D4fx8" # thay đổi bằng mật khẩu sử dụng trong neo4j


graph = Graph(uri, auth=(username, password))

graph

Graph('neo4j+s://73f9a4e9.databases.neo4j.io:7687')

## Remove the data    

In [4]:
def executeQuery(query):
    returnlist = []
    result = graph.run(query)
    for record in result:
        returnlist.append(record)
    
    return returnlist
            
    
def removeAllData():
    executeQuery("MATCH ()-[r]-() DELETE r;")
    executeQuery("MATCH (n) DELETE n;")
    print("Done! all data have been clear!")



In [5]:
#removeAllData()
executeQuery("MATCH(n) return n limit 25")

[]

## Import job data from csv to Neo4j
### Get the data from csv 

In [8]:
jobDataPath = DEFAULT_PATH + "job2024Clean.csv"
courseDataPath = DEFAULT_PATH + "courseClean.csv"

jobDataDf = pd.read_csv(jobDataPath)
courseDataDf = pd.read_csv(courseDataPath)
jobDataDf = jobDataDf.fillna("")
courseDataDf = courseDataDf.fillna("")

#jobDataDf.head(5)
courseDataDf.head(5) 

,Name,Link,Rating,Enroll,Instructor,Time,Level,skillrequirement,SkillWillLearn,Description,...,Platform,ProgramingLanguage,Practice Task,webname,weburl,NeedKnowledge,NeedPlatform,NeedFramework,NeedProgrammingLanguage,NeedTool
0,Bacula 1: the open source backup software,https://www.udemy.com/course/bacula-backup-sof...,4.2,697,Heitor Faria,8 hours on-demand video,,2 virtual or physical Linux machines for Bacul...,"Make the student ready to fully plan, install,...",The course presents the theory and practice of...,...,"bacula, bacula server",,"run and after backup scripts, installing and c...",udemy,www.udemy.com,,"LINUX,WINDOWS,BACULA",LINUX,,"LINUX,WINDOWS"
1,Learning Android Apps Development from Scratch,https://www.udemy.com/course/learning-android-...,4.0,0,Sarah Alenzei,2.5 hours on-demand video,,No Previous programming skills required,Develop Android App. Publish apps at google pl...,Start your Android Development journey here le...,...,google play,,"publish apps, publish their own apps, creating...",udemy,www.udemy.com,PROGRAMMING,,,,
2,Free Android Development Tutorial - How to pub...,https://www.udemy.com/course/how-to-publish-an...,4.5,55,Cristian Gradisteanu,32min,,,,Ever wanted to know what are the required step...,...,google play,,"update your previous published app, generate a...",udemy,www.udemy.com,,,,,
3,Bentley STAAD Pro - Structural Analysis & Desi...,https://www.udemy.com/course/staadpro-3d-struc...,4.2,1,SS eAcademy .,6 hours on-demand video,,Basic knowledge of Structural Analysis and Des...,Introduction of STAAD Pro and Structural Engin...,STAAD Pro stands for Structural Analysis and d...,...,,,"apply different type of load, modify beam and ...",udemy,www.udemy.com,"ANALYSIS,SOFTWARE",,,,"STRUCTURAL ANALYSIS AND DESIGN,STAAD PRO,AUTOC..."
4,Software Testing/ QA: Learn Basic Testing with...,https://www.udemy.com/course/software-qa-testi...,3.6,3,Syam Mohan,2 hours on-demand video,,"Nothing special, Any one can join!",Writing test cases. Strong testing skills. Goo...,NOTE: This is a software testing beginner cour...,...,,,"writing test cases, bug tracking, live debug",udemy,www.udemy.com,,,,,


---
### Time processing
---


In [9]:


def convertTime2Hours(time_str):
    if pd.isnull(time_str):
        return -1
    # Check for minutes in various formats and convert to hours
    minutes_match = re.search(r'(\d+)\s*(min|mins|minute|minutes|m)\b', time_str, re.IGNORECASE)
    if minutes_match:
        minutes = int(minutes_match.group(1))
        return round(minutes / 60.0, 2)
    # Extract hours
    hours_match = re.search(r'(\d+\.?\d*)\s*hours?', time_str, re.IGNORECASE)
    if hours_match:
        return round(float(hours_match.group(1)), 2)
    return -1



### Apply the time formatting

In [10]:
courseDataDf['Duration'] = courseDataDf['Time'].apply(convertTime2Hours)
courseDataDf.head(5) 

,Name,Link,Rating,Enroll,Instructor,Time,Level,skillrequirement,SkillWillLearn,Description,...,ProgramingLanguage,Practice Task,webname,weburl,NeedKnowledge,NeedPlatform,NeedFramework,NeedProgrammingLanguage,NeedTool,Duration
0,Bacula 1: the open source backup software,https://www.udemy.com/course/bacula-backup-sof...,4.2,697,Heitor Faria,8 hours on-demand video,,2 virtual or physical Linux machines for Bacul...,"Make the student ready to fully plan, install,...",The course presents the theory and practice of...,...,,"run and after backup scripts, installing and c...",udemy,www.udemy.com,,"LINUX,WINDOWS,BACULA",LINUX,,"LINUX,WINDOWS",8.00
1,Learning Android Apps Development from Scratch,https://www.udemy.com/course/learning-android-...,4.0,0,Sarah Alenzei,2.5 hours on-demand video,,No Previous programming skills required,Develop Android App. Publish apps at google pl...,Start your Android Development journey here le...,...,,"publish apps, publish their own apps, creating...",udemy,www.udemy.com,PROGRAMMING,,,,,2.50
2,Free Android Development Tutorial - How to pub...,https://www.udemy.com/course/how-to-publish-an...,4.5,55,Cristian Gradisteanu,32min,,,,Ever wanted to know what are the required step...,...,,"update your previous published app, generate a...",udemy,www.udemy.com,,,,,,0.53
3,Bentley STAAD Pro - Structural Analysis & Desi...,https://www.udemy.com/course/staadpro-3d-struc...,4.2,1,SS eAcademy .,6 hours on-demand video,,Basic knowledge of Structural Analysis and Des...,Introduction of STAAD Pro and Structural Engin...,STAAD Pro stands for Structural Analysis and d...,...,,"apply different type of load, modify beam and ...",udemy,www.udemy.com,"ANALYSIS,SOFTWARE",,,,"STRUCTURAL ANALYSIS AND DESIGN,STAAD PRO,AUTOC...",6.00
4,Software Testing/ QA: Learn Basic Testing with...,https://www.udemy.com/course/software-qa-testi...,3.6,3,Syam Mohan,2 hours on-demand video,,"Nothing special, Any one can join!",Writing test cases. Strong testing skills. Goo...,NOTE: This is a software testing beginner cour...,...,,"writing test cases, bug tracking, live debug",udemy,www.udemy.com,,,,,,2.00


## Import data to neo4j
### Support function 

---

### Create or merge Node functions
---

In [11]:

# create a new node without checking anything
def createNewNode(nodeName, propertiesList):
    properties = {k: v for k, v in propertiesList.items()}
    new_node = Node(nodeName, **properties)
    graph.create(new_node)
    return new_node

#----------------------------------------------------------------
# create new node if it does not exist else merge that node with single property
#----------------------------------------------------------------
def getOrCreateNode(nodeName, attribute, attributeValue, level=None):
    
    cypher_query = ""
    if (pd.isna(attributeValue)): return None

    # check if value checker is string of not
    if (type(attributeValue) == str): 
        if(len(attributeValue) < 2): return None
        
        attributeValue = attributeValue.replace("\"", "'")

        cypher_query = f"MATCH (n:{nodeName}) WHERE n.{attribute}=\"{attributeValue}\" RETURN n"
    else:
        cypher_query = f"MATCH (n:{nodeName}) WHERE n.{attribute}={attributeValue} RETURN n"

    result = graph.evaluate(cypher_query)
    if (result is None):
        newNode = Node(nodeName, **{attribute: attributeValue})
        result = graph.create(newNode)
        result = newNode
    

    return result


#----------------------------------------------------------------
# create new node if it does not exist else merge that node with multi properties
#----------------------------------------------------------------

def getOrCreateNodeMultiAttr(nodeName, attrChecker, AttrCheckerValue, propertiesList, level=None):
    # Construct the WHERE clause for the Cypher query
    if (pd.isna(AttrCheckerValue)): return None
    cypher_query =""
    if (type(AttrCheckerValue) == str): 
        if len(AttrCheckerValue) < 2:  return None
        propertiesList[attrChecker] = AttrCheckerValue.replace("\"", "'")
        AttrCheckerValue = AttrCheckerValue.replace("\"", "'")
        if level is not None:
            cypher_query = f"MATCH (n:{nodeName}) WHERE n.{attrChecker}=\"{AttrCheckerValue}\" and n.level = \"{level}\" RETURN n"
        else:
            if nodeName == "Framework" or nodeName == "ProgrammingLanguage":
                cypher_query = f"MATCH (n:{nodeName}) WHERE n.{attrChecker}=\"{AttrCheckerValue}\" and n.level is null RETURN n"
            else:
                cypher_query = f"MATCH (n:{nodeName}) WHERE n.{attrChecker}=\"{AttrCheckerValue}\" RETURN n"
    else:
        cypher_query = f"MATCH (n:{nodeName}) WHERE n.{attrChecker}={AttrCheckerValue} RETURN n"

    result = graph.evaluate(cypher_query)

    if result is None:
        # Dynamically set properties using keyword arguments
        properties = {k: v for k, v in propertiesList.items()}
        new_node = Node(nodeName, **properties)
        graph.create(new_node)
        return new_node

    return result

def makeRelationship(sourceNode, relation, targetNode):
    if (sourceNode is None or targetNode is None): return
    graph.merge(Relationship(sourceNode,relation, targetNode))




### Testing area

In [12]:
n = getOrCreateNode("Career", "name", "Hitachi\"boku\"")
m = getOrCreateNode("Career", "name", "Hitachi\"boku\"")

executeQuery("MATCH(n) return n limit 25")
#graph.delete_all()


[Record({'n': Node('Career', name="Hitachi'boku'")})]

---
### Make node and relationship of Competency
---

In [13]:



# create or megre a comptency node and set relationship between it and source node
def competencyProcess(sourceNode, competency, competencyValue, relationName, level=None):

    if (pd.isna(competencyValue) or len(competencyValue) < 1): return

    nodeName = competency # Node to process
    nodeAttr = competency[:1].lower() + competency[1:] # lower case for first character
    
    # separate competency to each single value
    for comp in competencyValue.split(','):
        if (len(comp) < 2): continue
        
        # make the property 
        property = {nodeAttr: comp.strip().upper()}

        # if level is not none, add level to properties
        if level is not None:
            level = level.upper()
            property["level"] = level

        compe = getOrCreateNodeMultiAttr(nodeName, nodeAttr, comp.strip().upper(), property, level)

        makeRelationship(sourceNode, relationName, compe)



def processAllCompetency(Knowledge, ProgrammingLanguage, Framework, Platform, Tool, level=None):

    # check if competency of this row is empty or not, if not then do nothing
    if (pd.isna(ProgrammingLanguage) or len(ProgrammingLanguage) < 1): return

    # separate competencies into single value
    for program in ProgrammingLanguage.split(','):
        if (len(program) < 2): continue
         # make the property 
        property = {"programmingLanguage": program.strip().upper()}

        # if level is not none, add level to properties
        if level is not None:
            level = level.upper()
            property["level"] = level
        Program = getOrCreateNodeMultiAttr("ProgrammingLanguage", "programmingLanguage", program.strip().upper(), property, level)
        competencyProcess(Program, "Knowledge", Knowledge, "Relate_to_knowledge")
        competencyProcess(Program, "Framework", Framework, "Have_framework", level)
        competencyProcess(Program, "Tool", Tool, "Use_tool") 

    if (pd.isna(Framework) or len(Framework) < 1): return

    for frame in Framework.split(','):
        if (len(frame) < 2): continue
        property = {"framework": frame.strip().upper()}

        # if level is not none, add level to properties
        if level is not None:
            property["level"] = level.upper()

        frameNode = getOrCreateNodeMultiAttr("Framework", "framework", frame.strip().upper(), property , level)
        competencyProcess(frameNode, "Platform", Platform, "Deploy_to_platform")
        competencyProcess(frameNode, "Knowledge", Knowledge, "Relate_to_framework")

### Testing area

In [14]:
graph.delete_all()
processAllCompetency("Data analysis, Databases , Data warehouse", "JAVA , Python, C#","Spark", "MySQL, Oracle", "GIT", "basic")
processAllCompetency("Data analysis, Databases , Data warehouse", "JAVA , Python, C#","Spark", "MySQL, Oracle", "GIT")
processAllCompetency("Data analysis, Databases , Data warehouse", "JAVA , Python, C#","Spark", "MySQL, Oracle", "GIT", "Advanced")


executeQuery("MATCH(n) return n limit 25")

[Record({'n': Node('Knowledge', knowledge='DATA ANALYSIS')}),
 Record({'n': Node('ProgrammingLanguage', level='BASIC', programmingLanguage='JAVA')}),
 Record({'n': Node('Knowledge', knowledge='DATABASES')}),
 Record({'n': Node('Knowledge', knowledge='DATA WAREHOUSE')}),
 Record({'n': Node('Framework', framework='SPARK', level='BASIC')}),
 Record({'n': Node('Tool', tool='GIT')}),
 Record({'n': Node('ProgrammingLanguage', level='BASIC', programmingLanguage='PYTHON')}),
 Record({'n': Node('ProgrammingLanguage', level='BASIC', programmingLanguage='C#')}),
 Record({'n': Node('Platform', platform='MYSQL')}),
 Record({'n': Node('Platform', platform='ORACLE')}),
 Record({'n': Node('ProgrammingLanguage', programmingLanguage='JAVA')}),
 Record({'n': Node('Framework', framework='SPARK')}),
 Record({'n': Node('ProgrammingLanguage', programmingLanguage='PYTHON')}),
 Record({'n': Node('ProgrammingLanguage', programmingLanguage='C#')}),
 Record({'n': Node('ProgrammingLanguage', level='ADVANCED', prog

---
### Make node and relationship of Time (Date, Month, Year)
---

In [15]:

def getOrCreatetMonthNode(month, year):
    
    cQuery = f"MATCH (m: Month {{month:{month}}})-[:part_of]->(y: Year {{year:{year}}})  RETURN m"
    result = graph.evaluate(cQuery)

    if result is None:
        Year = getOrCreateNode("Year", "year", year)
        Month = Node("Month", month=month)
        graph.create(Month)
        graph.merge(Relationship(Month, "part_of", Year))
        return Month

    return result



def getOrCreatetDateNode(date, month, year):

    # Construct the WHERE clause for the Cypher query
    cypher_query = f"MATCH (n: Date {{day:{date}}})-[:part_of]->(m: Month {{month:{month}}})-[:part_of]->(y: Year {{year:{year}}})  RETURN n"
    
    result = graph.evaluate(cypher_query)

    if result is None:
        Month = getOrCreatetMonthNode(month, year)
        Date = Node("Date", day=date)
        graph.create(Date)
        graph.merge(Relationship(Date, "part_of", Month))
        return Date

    return result


---
#### Testing Area --------------------------------
---

In [16]:

# result = getOrCreatetDateNode(14,1,1999)
# result = getOrCreatetDateNode(14,2,1999)
# result = getOrCreatetDateNode(24,1,1999)
# result = getOrCreatetDateNode(24,1,1999)
# result = getOrCreatetDateNode(24,1,1999)

# executeQuery("MATCH(n) return n limit 25")
graph.delete_all()

# Import Course data

## Nguyên tắc
Name,Link,Rating,Enroll,Instructor,Time,Level,skillrequirement,SkillWillLearn,Description,SkillGain,Subject,Organization,Price,Subtitle,Knowledge,Tool,Framework,Platform,ProgramingLanguage,Practice Task,webname,weburl,NeedKnowledge,NeedPlatform,NeedFramework,NeedProgrammingLanguage,NeedTool

### Chia thành 3 nhóm: Fact, Competency, Course related. Và gồm 1 + 5 + 6  = 12 thực thể liên kết  với nhau
**Fact**: Bao gồm FactCourse (Enroll, rating) \
***Fact Relationship***:
- Fact - Course: belong_to_course
- Fact - Competency: teach_competency
- Fact - Website: post_on
- Fact - Ozganization: belong_to

**Course**: Bao gồm Course(name, link, description, subject, price, duration, skillWillLearn), Instructor(name), <br> Website(name, url), Level(level), Organization(name), Subtitle(language) \
***Fact Relationship***:
- Course - Instructor: teach_by
- Course - Competency: teach_competency
- Course - Level: have_level
- Course - Subtitle: have_subtitle

**Competency**: Bao gồm Knowledge (knowledge), ProgrammingLanguage(programmingLanguage),<br> Platform(platform), Framework(framework), Tool(tool) \
***Competency Relationship***:
- Knowledge - ProgrammingLanguage, Platform, Framework, Tool: relate_to
- ProgramingLanguage - Framework: have_framework
- ProgrammingLanguage - Tool: have_tool
- Platform - Framework: deploy_on

In [17]:
def levelRequire(Level):

    if Level is None or Level == "Basic" :
        return None
    elif Level == "Advanced" :
        return "Intermedidate"
    else:
        return "Basic"

    return None



In [18]:



def importCourseData(courseData):
    
    mileStone = len(courseData)
    i = 0
    for index, row in courseData.iterrows():

        # create fact group
        if (pd.isna(row['Enroll'])): continue
        
        FactCourse = createNewNode("FactCourse", {"enroll": row['Enroll'], "rating": row['Rating']})
        
        levelVal = row['Level']
        if (levelVal == "Beginner"): levelVal = "Basic"
        levelVal = levelVal.upper()
        # create compentency 
        #(teached competency)
        processAllCompetency(row['Knowledge'], row['ProgramingLanguage'], row['Framework'], row['Platform'], row['Tool'], levelVal)
        competencyProcess(FactCourse, "Knowledge", row['Knowledge'], "Taught_knowledge" )
        competencyProcess(FactCourse, "ProgrammingLanguage", row['ProgramingLanguage'], "Taught_programmingLanguage", levelVal)
        competencyProcess(FactCourse, "Framework", row['Framework'], "Taught_framework" , levelVal)
        competencyProcess(FactCourse, "Platform", row['Platform'], "Taught_platform" )
        competencyProcess(FactCourse, "Tool", row['Tool'], "Taught_tool")


        # Create course related group 
        courseRow = {"name": row['Name'], "description": row['Description'], "link": row['Link'], "subject": row['Subject'],
                      "price": row['Price'], "skillWillLearn": row['SkillWillLearn'], "duration": row['Duration'], "level": levelVal}
        
        Course = getOrCreateNodeMultiAttr("Course", "link", row['Link'], courseRow)
        Organization = getOrCreateNode("Organization", "name", row['Organization'])

        
        
        Instructor = getOrCreateNode("Instructor", "name", row['Instructor'])
        Website = getOrCreateNodeMultiAttr("Website", "name", row['webname'], {"name": row['webname'], "url": row['weburl']})
        if not pd.isna(row['Subtitle']):
            for sub in row['Subtitle'].split(","):
                if (len(sub) < 2): continue
                Subtitle = getOrCreateNode("Subtitle", "language", sub.strip().upper())
                makeRelationship(Course, "Have_subtitle", Subtitle)

        #(required competency (Pre-skill need to learn))
        competencyProcess(Course, "Knowledge", row['NeedKnowledge'], "Nead_knowledge" )
        competencyProcess(Course, "ProgrammingLanguage", row['NeedProgrammingLanguage'], "Need_programmingLanguage", levelRequire(levelVal).upper() )
        competencyProcess(Course, "Framework", row['NeedFramework'], "Need_framework", levelRequire(levelVal).upper() )
        competencyProcess(Course, "Platform", row['NeedPlatform'], "Need_platform" )
        competencyProcess(Course, "Tool", row['NeedTool'], "Need_tool")

        # makeRelationship(Course, "Had_level", Level)
        makeRelationship(Course, "Taught_by", Instructor)
        makeRelationship(FactCourse, "Belong_to", Organization)
        makeRelationship(FactCourse, "Posted_on", Website)
        makeRelationship(FactCourse, "Belong_to_course", Course)
        i = i + 1
        print(f"Loading process {i/mileStone * 100} %", end='\r')

    print("\nDone!")

        




In [19]:

graph.delete_all()
importCourseData(courseDataDf)


Node('ProgrammingLanguage', ProgrammingLanguage='Java', level='Intermediate')

In [48]:


executeQuery("MATCH(n) return n limit 25")


[Record({'n': Node('Knowledge', knowledge='SOFTWARE ARCHITECTURE')}),
 Record({'n': Node('Course', description='20 years ago I graduated with a first class software engineering degree. Since then I\'ve worked at every stage of the software lifecycle butI\'ve always had a particularinterest in software architecture. If you have a software related background and you\'re interested in becoming a software architect, this course is definitely for you. Erik N says: "The course provided a good overview of the topic for which I am interested, namely, software architecture. The content gave me some good ideas, encouragement and new avenues to explore. Sami S says: "Good starting point to Software Architecture, at least to me who has+ years of software developer background. Jose V says: "It\'s a good introductory course, very digestible and serves the purpose. Balint E says: "It\'s a good basic introduction to the field. In this course I\'ll teach you what software architecture is and why we nee

# Import job data

In [73]:
def importJobData(df):
    # Iterate through rows and create nodes and relationships
    mileStone = len(df)
    i = 0
    for index, row in df.iterrows():
            
        Organization = getOrCreateNode("Organization", "name", row['companyName'])
        Location = getOrCreateNode( "Location", "location", row['location'])
      
        webName = row['website']
        webUrl =  "https://www." + webName + ".com"
        Website = getOrCreateNodeMultiAttr("Website", "name", webName, {"name": webName, "url": webUrl})

        # Create Career node or retrieve existing one
        Career = getOrCreateNode( "Career", "name", row['jobType'])
    
        # Create FactJobPosting node or retrieve existing one
        FactJobPosting = Node("FactJobPosting", totalJobPost=row['jobCount'])
        graph.create(FactJobPosting)

        # Create Date node or retrieve existing one
        timePost = getOrCreatetDateNode(row['date'],row['month'], row['year'])
        
        makeRelationship(FactJobPosting, "Recruited_by", Organization)
        makeRelationship(FactJobPosting, "Published_on", Website)
        makeRelationship(FactJobPosting, "Belong_to_career", Career)
        makeRelationship(FactJobPosting, "Located_at", Location)
        makeRelationship(FactJobPosting, "Posted_at_time", timePost)
        
        competencyProcess(FactJobPosting, "Knowledge", row['Knowledge'], "Required_knowledge" )
        competencyProcess(FactJobPosting, "ProgrammingLanguage", row['ProgrammingLanguage'], "Required_programmingLanguage" )
        competencyProcess(FactJobPosting, "Framework", row['Framework'], "Required_framework" )
        competencyProcess(FactJobPosting, "Platform", row['Platform'], "Required_platform" )
        competencyProcess(FactJobPosting, "Tool", row['Tool'], "Required_tool")

        i = i + 1
        print(f"Loading process {i/mileStone * 100} %", end='\r')
        


In [74]:
importJobData(jobDataDf)

---
### Load user 
---


## radom user function

In [15]:
import random
import csv
from datetime import datetime, timedelta

# Provided data
lastName = ["Viet", "Vu", "Sang", "Thi", "Thuy", "Huong", "Dang"]
middleName = ["Van", "Thi"]
firstName = ["Nguyen", "Tran", "Le", "Phan"]
userSkill = {
    "ProgrammingLanguage": ["Java", "Python", "C#"],
    "Framework": ["Pandas", "Spark", "Spring", "ASP.NET"],
    "Tool": ["Git", "MySQL", "ORACLE DATABASE"],
    "Knowledge": ["Analysis", "Database", "OOP", "Algorithm", "DATA ANALYSIS", "DATA WAREHOUSE"],
    "Platform": ["Azure", "Cloud", "MySQL"]
}
userSkill = {key: [skill.upper() for skill in value] for key, value in userSkill.items()}

def generate_random_date(start_year=1996, end_year=2010):
    # Generate a random date between January 1, start_year and January 1, end_year
    start_date = datetime(year=start_year, month=1, day=1)
    end_date = datetime(year=end_year, month=1, day=1)
    time_between_dates = end_date - start_date
    days_between_dates = time_between_dates.days
    random_number_of_days = random.randrange(days_between_dates)
    random_date = start_date + timedelta(days=random_number_of_days)  # Corrected timedelta usage
    return random_date.strftime('%m/%d/%Y')  # Formatting date as MM/DD/YYYY




#----------------------------------------------------------------
# the users are all random -> so, run this function only once
# and just read the csv file to load -> ensure the consistency 
#----------------------------------------------------------------


#Prepare the CSV file and writer
# with open('userData.csv', 'w', newline='') as file:
#     writer = csv.writer(file)
#     # Write the header row
#     writer.writerow(['Username','FullName', 'Email', 'DateOfBirth', 'Phone', 'ProgrammingLanguage', 'Framework', 'Tool', 'Knowledge', 'Platform'])

#     for i in range(1, 21):
#         fullName = f"{random.choice(firstName)} {random.choice(middleName)} {random.choice(lastName)}"
#         username = "User0" + str(i)
#         email = f"{fullName.replace(' ', '').lower()}{random.randint(100, 999)}@gmail.com"
#         dateOfBirth = generate_random_date()
#         phone = f"+84{random.randint(100000000, 999999999)}"
#         # Extracting all skills as comma-separated strings
#         ProgrammingLanguage = ', '.join(userSkill['ProgrammingLanguage'])
#         Framework = ', '.join(userSkill['Framework'])
#         Tool = ', '.join(userSkill['Tool'])
#         Knowledge = ', '.join(userSkill['Knowledge'])
#         Platform = ', '.join(userSkill['Platform'])

#         # Write the user data row
#         writer.writerow([username, fullName, email, dateOfBirth, phone, ProgrammingLanguage, Framework, Tool, Knowledge, Platform])


print("Write successful")


Write successful


In [78]:
userData = pd.read_csv(DEFAULT_PATH + "userData.csv")
len(userData)

20

In [34]:


for index, user in userData.iterrows():

    userData = {"username": user["Username"], "name": user["FullName"], "email": user["Email"], "dateOfBirth": user["DateOfBirth"], "phone": user["Phone"]}
    userNode = createNewNode("User", userData)

    for com in COMPETENCIES_LIST:
        nodeAttr = com[:1].lower() + com[1:] 
        for data in user[com].split(","):
            competencyProcess(userNode, com, data.strip() , "Known_" +  nodeAttr)




To delete all the users and their relationships
match(u:User)-[r]->() delete r,u

## Some query to analyze
- query1: match(f:FactJobPosting)-[:require_programmingLanguage]->(p:ProgrammingLanguage) return p.programmingLanguage as `Programming Language`, count(f) as `number of job`
- query2: match(f:FactJobPosting)-[:belong_to_career]->(p:Career) return p.name as Career, count(f) as `number of job`
- query3: match(f:FactJobPosting)-[:post_at_time]->(d:Date)-[:part_of]->(m:Month)-[:part_of]->(y:Year)return m.month as Month,y.year as Year, count(f) as `number of job`

In [ ]:
query = f'''MATCH (p:ProgrammingLanguage)-[r:Have_framework]->(fw:Framework)
optional match(p2:ProgrammingLanguage) where p2.programmingLanguage = p.programmingLanguage 
optional match(fw2) where fw2.framework = fw.framework
with p2, fw2, p, fw
where p2.level = fw2.level
merge(p2)-[:Have_framework]->(fw2) '''

query2 = f'''MATCH (p:ProgrammingLanguage)-[:Relate_to_knowledge]->(k:Knowledge)
optional match(p2:ProgrammingLanguage) where p2.programmingLanguage = p.programmingLanguage 
merge(p2)-[:Relate_to_knowledge]->(k)'''

query3 = f'''MATCH (p:ProgrammingLanguage)-[:Use_tool]->(t:Tool)
optional match(p2:ProgrammingLanguage) where p2.programmingLanguage = p.programmingLanguage 
merge(p2)-[:Use_tool]->(t)'''

query4 = f'''MATCH (f:Framework)-[:Deploy_to_platform]->(p: Platform)
optional match(fw2) where fw2.framework = f.framework
merge(fw2)-[:Deploy_to_platform]->(p)'''

query5 = f'''match(u: User) where u.username in ["User01", "User02", "User03"]
match(k:Knowledge)
where k.knowledge in ["DATABASES", "SQL", "DATA PIPELINES", "COMPUTER SCIENCE"]
merge (u)-[:Known_knowledge]->(k)'''